# Accessing a Generative AI Model through OpenAI API

Version: 2025-10-13

Generative AI models are machine learning models that are capable of generating content.
This notebook is going to focus on the most commonly-used type of generative AI models&mdash;the 
ones that generate text. These models are called *large language models*, or 'LLM' for short.
We will not be going into the technical details of how a LLM works at this moment, but rather focus
on how to use an LLM as an end user.

## A. Accessing an LLM through Python
Although it is very convenient to access an LLM through a browser interface for one-off task, 
for repetitive task you will generating want to access the model using a program. Model providers 
will generally provide such access through an *application programming interface*, or 'API' for short.
An API defines what sorts of interactions are possible with the model, and how input and output data 
should look like. 

The API used by OpenAI, the provider of ChatGPT, is the de facto standard supported by most models.

The first step is to set up an OpenAI client. The client requires two piece of information:
1. `base_url`: this is the web address of the model provider. If you use OpenAI's model, this can be omited.
2. `api_key`: this is a string of text unique to your user account with the model provider. 

In [1]:
from openai import OpenAI

client = OpenAI(
    base_url = 'https://scrp-chat.econ.cuhk.edu.hk/api',
    api_key='your_api_key_here',
)

Note that whoever has your API key has access to your AI service account. In a production environment, it is therefore best to save the API key as an environment variable, so that it will not be saved in your notebook. 

The easiest way to do so when working in a Jupyter notebook is to use the `dotenv` library to load environment variables from a file. The library defaults to loading the variables from a file named `.env` in your current working directory, but you can load from any file by providing a suitable path. 

In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import os

# Load the scrp-chat.env file containing environment variables.
# Default is to look for a file named .env in the current working directory.
home_dir = os.environ["HOME"]
load_dotenv(os.path.join(home_dir,"scrp-chat.env"))

# The API key should be saved as an environment variable OPENAI_API_KEY
# The base_url should be saved as OPENAI_BASE_URL in the same file
client = OpenAI()

## B. Model List

To see what models the provider support:

In [ ]:
models = client.models.list()
print("Available Models:")
for model in models:
    print(model.id)

In [3]:
models = client.models.list()
print("Available Models:")
for model in models:
    print(model.id)

Available Models:
Qwen/Qwen2.5-VL-72B-Instruct-AWQ
Qwen/Qwen3-32B-FP8
abc-test
default
help
image-editor-function
openai/gpt-oss-120b
openai/gpt-oss-120b
reasoning
text
text-large
text-medium
text-small
vision
vision-large
vision-medium
vision-small
zai-org/GLM-4.5-FP8


## C. Calling the Model

To call a model, use 
```python
response = client.chat.completions.create(...)
```
You will need to provide a few pieces of information:
- `model`: the name of the model.
- `messages`: a list containing the conversation history. Each message should be dictionary with a `role` and a `content`. `role` can be one of:
    - `system`: system prompt. Use this to give the model general guidelines that it should follow strictly.
    - `user`: prompt entered by the user. 
    - `assistant`: response from the model. 
    - `tool`: response from tools the model can use.
- Additional settings such as temperature and number of samples:
    - `temperature`: Adjust the randomness of the response. 0 is the lowest setting.
      Default is model specfic, around 0.6-0.8.
    - `n`: number of responses you want the model to generate. Default is 1.

The response will be recorded in
```python
response.choices[0].message.content
```

In [4]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="default",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Where is Hong Kong?"},
  ],
)
print(response.choices[0].message.content)

Hong Kong is a Special Administrative Region (SAR) on the southern coast of China.  

- **Geographic setting:** It sits on the eastern side of the Pearl River Delta, bordering the city of Shenzhen in Guangdong Province to the north and facing the South China Sea to the south, east, and west.  
- **Coordinates:** Approximately **22.3° N latitude, 114.2° E longitude**.  
- **Nearby major cities:**  
  - About **30 km (≈ 19 mi)** north of the border with Shenzhen.  
  - Roughly **140 km (≈ 87 mi)** southwest of Guangzhou.  
  - Approximately **2,400 km (≈ 1,490 mi)** southeast of Beijing.  

Politically, Hong Kong is part of the People’s Republic of China but operates under the “one‑country, two‑systems” framework, giving it a high degree of autonomy and its own legal and economic systems.


## D. Example: Scrapping Webpage and Extract Information with LLM

We first scrape the webpage with Selenium + Beautiful Soup,
but unlike before, we do not manually find the elements we need.
Instead, we export a stripped version of the webpage's source code,
then uses an LLM to extract the information we need. 

Because LLMs can only process a limited amount of text input, called 
*context length*, and slows down as the context length goes up, 
it is usually wise to use Beautiful Soup to locate
as precious as possible what you need, before passing the source code
to the LLM.

### D1. Fetch the Page Source in Full

Scrape the webpage in full:

In [5]:
# Only works with static content.
# See A6-Data-Scrping-Completed for a version that works with dynamic content.
import requests
def get_page_source(url):
    page = requests.get(url)
    return page.text

Now we feed the source code to the LLM:

In [6]:
from openai import OpenAI
import time

page_source = get_page_source('http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20151213/ST')

client = OpenAI()

start = time.time()
response = client.chat.completions.create(
  model="default",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": f"""The following webpage contains horse racing results. 
                                Please extract the result table and return it in comma-delimited
                                CSV format. No explanation is needed.
                                {page_source}
                                """},
  ],
)
output = response.choices[0].message.content
print("Processing took: {:.2f}".format(time.time() - start))
print(output)

Processing took: 14.49
Place,Horse No,Horse,Jockey,Trainer,Act. Wt.,Declar. Horse Wt.,Draw,LBW,Running Position,Finish Time,Win Odds
1,10,"JOLLY JOLLY (T087)","K Teetan","P O'Sullivan",114,1214,13,"-", "1 1 1 1",1:22.05,2.6
2,8,"PEOPLE'S KNIGHT (T305)","T Berry","J Moore",119,1163,8,2,"2 4 2 2",1:22.39,5.7
3,12,"RUN FORREST (T176)","J Moreira","C S Shum",115,1135,10,3,"14 10 10 3",1:22.54,3.9
4,4,"MODERN TSAR (S167)","B Prebble","W Y So",123,1101,11,"4‑3/4","9 13 13 4",1:22.80,13
5,3,"MAGNETISM (V114)","G Lerena","D E Ferraris",125,1130,3,"4‑3/4","7 6 6 5",1:22.81,52
6,1,"ENORMOUS HONOUR (T236)","N Rawiller","Y S Tsui",131,1127,9,"5‑1/4","11 12 12 6",1:22.87,10
7,9,"HAPPY JOURNEY (S299)","H W Lai","S Woods",114,1040,5,"6‑1/2","5 7 7 7",1:23.08,121
8,14,"WINGOLD (T202)","M L Yeung","A Lee",111,1154,12,"6‑1/2","13 14 14 8",1:23.11,331
9,11,"OVETT (P351)","H N Wong","A T Millard",105,1153,4,"7‑1/4","3 2 3 9",1:23.20,41
10,5,"PAKISTAN BABY (S442)","D Whyte","A S Cruz",121,1023,7,"7‑1/4","6

### D2. Strip the Page Source

You can strip white space by setting `strip = True` in `get_page_source()`, 
but it appears to drastically lower the quality of the LLM's output.
The likely reason for this is that during training, the LLM has learnt to 
use the leading and trailing white space in understanding the content. 
Removing the whitespace therefore make the content harder to understand,
just like it would be for an actual human.

In [7]:
# Page source with white space stripped
import requests
from bs4 import BeautifulSoup

def get_stripped_page_source(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    return soup.get_text(strip=True)   

Let us take a look at the stripped source code:

In [8]:
page_source = get_stripped_page_source('http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20151213/ST')
page_source[0:500]

"Results - Racing Information - Horse Racing - The Hong Kong Jockey ClubHome中文Login/RegisterFAQHorse RacingFootballMark SixRacing Info (Local)|EntriesRace CardTrackworkPast Incidents ExtractForm Line ReportExceptional FactorsVeterinary RecordsRace CardCurrent OddsTrainers' Entries for the DayTNC Entries ListTrainers' EntriesJockeys' Rides For The DayJKC Scheduled Rides ListJockeys' RidesChangesGeneral InformationRaceday FocusSpeedPROStats CentreLast Run ReminderIntro to New HorsesHorse Movement R"

Now let us provide the page source to the LLM:

In [9]:
page_source = get_stripped_page_source('http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20151213/ST')
start = time.time()
response = client.chat.completions.create(
  model="default",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": f"""The following webpage contains horse racing results. 
                                Please extract the result table and return it in comma-delimited
                                CSV format. No explanation is needed.
                                {page_source}
                                """},
  ],
)
output = response.choices[0].message.content
print("Processing took: {:.2f}".format(time.time() - start))
print(output)

Processing took: 18.33
Placement,Horse No,Horse,Jockey,Trainer,Act Wt,Declar. Horse Wt,Draw,LBW,Running Position,Finish Time,Win Odds
1,110,"JOLLY JOLLY (T087)","K Teetan","P O'Sullivan",114,121,413,-111,1,1:22.05,2.62
2,119,"PEOPLE'S KNIGHT (T305)","T Berry","J Moore",119,116,382,242,2,1:22.39,5.73
3,115,"RUN FORREST (T176)","J Moreira","C S Shum",115,113,510,314,10,1:22.54,3.94
4,123,"MODERN TSAR (S167)","B Prebble","W Y So",123,110,111,"4-3/49",13,1:22.80,13.53
5,125,"MAGNETISM (V114)","G Lerena","D E Ferraris",125,113,34,"-3/47",66,1:22.81,5.26
6,131,"ENORMOUS HONOUR (T236)","N Rawiller","Y S Tsui",131,112,79,"-1/41",11,1:22.87,10.79
7,114,"HAPPY JOURNEY (S299)","H W Lai","S Woods",114,104,56,"-1/25",7,1:23.08,12.18
8,111,"WINGOLD (T202)","M L Yeung","A Lee",111,115,41,26,"-1/21",31,1:23.11,13.31
9,105,"OVETT (P351)","H N Wong","A T Millard",105,115,34,7,"-1/43",23,1:23.20,41.00
10,121,"PAKISTAN BABY (S442)","D Whyte","A S Cruz",121,102,37,7,"-1/46",11,1:23.22,11.10
11,120,"SUPER F

### D3. Target Specific Elements

To speed up the processing and make the model's output more accurate, you can use Beauitful Soup to locate the table, and provide only the source code of the table to the LLM:

In [10]:
import requests
def get_table_source(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    # Get the result table
    result_table = soup.find("table", class_="f_tac")
    if result_table:
        return result_table.prettify()
    else:
        return None

Let us try again:

In [11]:
page_source = get_table_source('http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20151213/ST')
start = time.time()
response = client.chat.completions.create(
  model="default",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": f"""The following webpage contains horse racing results. 
                                Please extract the result table and return it in comma-delimited
                                CSV format. No explanation is needed.
                                {page_source}
                                """},
  ],
)
output = response.choices[0].message.content
print("Processing took: {:.2f}".format(time.time() - start))
print(output)

Processing took: 10.67
Pla.,Horse No.,Horse,Jockey,Trainer,Act. Wt.,Declar. Horse Wt.,Dr.,LBW,Running Position,Finish Time,Win Odds
1,10,JOLLY JOLLY,K Teetan,P O'Sullivan,114,1214,13,,1 1 1 1,1:22.05,2.6
2,8,PEOPLE'S KNIGHT,T Berry,J Moore,119,1163,8,2,2 4 2 2,1:22.39,5.7
3,12,RUN FORREST,J Moreira,C S Shum,115,1135,10,3,14 10 10 3,1:22.54,3.9
4,4,MODERN TSAR,B Prebble,W Y So,123,1101,11,4-3/4,9 13 13 4,1:22.80,13
5,3,MAGNETISM,G Lerena,D E Ferraris,125,1130,3,4-3/4,7 6 6 5,1:22.81,52
6,1,ENORMOUS HONOUR,N Rawiller,Y S Tsui,131,1127,9,5-1/4,11 12 12 6,1:22.87,10
7,9,HAPPY JOURNEY,H W Lai,S Woods,114,1040,5,6-1/2,5 7 7 7,1:23.08,121
8,14,WINGOLD,M L Yeung,A Lee,111,1154,12,6-1/2,13 14 14 8,1:23.11,331
9,11,OVETT,H N Wong,A T Millard,105,1153,4,7-1/4,3 2 3 9,1:23.20,41
10,5,PAKISTAN BABY,D Whyte,A S Cruz,121,1023,7,7-1/4,6 11 11 10,1:23.22,12
11,6,SUPER FLUKE,M Demuro,D Cruz,120,1109,14,7-3/4,4 3 4 11,1:23.27,65
12,7,JUN GONG,C Y Ho,C H Yip,115,1147,6,10-1/4,10 9 8 12,1:23.68,83
13,2,LAU

## E. Outputing to File

Because the LLM has done the formatting, we directly write
the output to file:

In [12]:
with open("result.csv", "w") as f:
  f.write(output)

## F. Scrape Multiple Pages

We can put everything together in one function
and use it in a loop to save the information we need:

In [15]:
# Function to scrape results

def scrape_webpage(url, prompt):
    output = ""
    page_source = get_table_source(url)
    if page_source is not None:
        response = client.chat.completions.create(
          model="default",
          messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt + page_source},
          ],
        )
        output = response.choices[0].message.content
    return output      

In [16]:
# The first part of the URL of data source
url_front = "http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/"
prompt = """The following webpage may contain horse racing results in a table. 
            If it does, please extract the result table and return it in 
            comma-delimited CSV format. If not, just return an empty string.
            No explanation is needed.
            """

# Copy the loop from above and incorporate the csv-saving code
for year in range(2017,2018):
    for month in range(1,2):
        for day in range(1,15):
            
            # Convert month and day to 2-digit representation
            month_2d = '{:02d}'.format(month)
            day_2d = '{:02d}'.format(day)
            
            # Full URL of data source
            url = url_front + str(year) + month_2d + day_2d
            
            # Print the URL so we know the progress so far
            print("Trying:",url)
            
            # Call our function to fetch and process data given the URL
            output = scrape_webpage(url, prompt)
            
            # Only save if there is something in content
            if len(output) > 0:
                filepath = str(year)+month_2d+day_2d+".csv"
                
                # Save to file
                with open(filepath, "w") as f:
                    f.write(output)
                    print(f"{filepath} saved.")



Trying: http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20170101
20170101.csv saved.
Trying: http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20170102
Trying: http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20170103
Trying: http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20170104
20170104.csv saved.
Trying: http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20170105
Trying: http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20170106
Trying: http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20170107
Trying: http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20170108
20170108.csv saved.
Trying: http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20170109
Trying: http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20170110
Trying: http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20170111
20170111.csv saved.
Trying: http://rac